In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/llm-essays/test_essays.csv
/kaggle/input/llm-essays/train_essays.csv
/kaggle/input/llmessays/essays_a.csv
/kaggle/input/llmessays/essays_d.csv
/kaggle/input/llmessays/essays_b.csv
/kaggle/input/llmessays/essays_c.csv
/kaggle/input/train-v2/train_v2_drcat_02.csv


In [2]:
import nltk
import subprocess

from nltk.corpus import wordnet
import tensorflow as tf

device_name = tf.test.gpu_device_name()

#Check for GPU
if "GPU" not in device_name:
    print("GPU device not found")
    
print('Found GPU at: {}'.format(device_name))

print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")


GPU device not found
Found GPU at: 
GPU not available :(


Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
import random
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


In [4]:
#Functions
def get_synonyms(word):
    """
    Function for sunonyms.

    Args
    word (str): The word for which we want to find a sononym.

    Returns
    list: A list containing the synonyms.
    """
    synonyms = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    return synonyms

def replace_with_synonym(sentence):
    """
    Fucntion to replace random words of a sentence with synonyms.

    Args
    sentence (str): The sentence for which we want to replace some of its words with synonyms. 

    Retruns
    str: the sentence with the new words.
    """
    tokens = word_tokenize(sentence)
    for i in range(len(tokens)):
        word = tokens[i]
        if word.isalpha():  # Process only alphabetical words
            synonyms = get_synonyms(word)
            if random.random()<0.4 and len(synonyms) != 0:
                # Choose a random synonym to replace the word
                new_word = random.choice(synonyms)
                tokens[i] = new_word
    return ' '.join(tokens)

def delete_random_words(sentence, deletion_probability=0.05):
    """
    Function which deletes some words of a sentence.

    Args
    sentence (str): the sentence from which we want to delete words.
    deletion_probability (float): The probability that each token will be deleted.

    Returns
    str: The reformed sentence.
    """
    tokens = word_tokenize(sentence)
    result_tokens = []

    for token in tokens:
        if random.random() > deletion_probability:
            result_tokens.append(token)

    return ' '.join(result_tokens)

def swap_random_words(sentence, prob = 0.1):
    """
    Function that swaps words within the sentence.

    Args
    sentence (str): the sentence in which we will swap words.
    prob (float): the probability that each token will be swapped.

    Returns
    str: the reformed sentence
    """
    tokens = word_tokenize(sentence)

    # Perform random swaps
    for i in range(len(tokens)):
        if random.random() < prob:  # 20% chance of swapping each word
            j = random.randint(0, len(tokens) - 1)
            x = tokens[i] 
            tokens[i] = tokens[j]
            tokens[j] = x
    return ' '.join(tokens)

def shuffle_sentences(text):
    """
    Function to shuffle the sentences of a text.

    Args
    text (str): the text that we want to shuffle its sentences.

    Returns
    str: the shuffled text
    """
    sentences = sent_tokenize(text)
    random.shuffle(sentences)
    return ' '.join(sentences)

def data_augment_general(sentence):
    if random.random() < 0.8:
        sentence = delete_random_words(sentence)
    
    sentence = swap_random_words(sentence)
    return sentence

def POS(word:str):
    if nltk.pos_tag([word])[0][1][0].lower() in ['v','n','r']:
        return nltk.pos_tag([word])[0][1][0].lower()
    elif nltk.pos_tag([word])[0][1][0].lower() == 'j':
        return 'a'
    else:
        return "n"

# Text Cleaning: Delete Special Characters from the Text

In this step, we will clean the text data by removing any special characters, numbers, and punctuations. This is essential to ensure that the text data is in a standardised format, which will help in better analysis and model training.


In [5]:
dt = pd.read_csv("/kaggle/input/train-v2/train_v2_drcat_02.csv").drop(['prompt_name',"source","RDizzl3_seven"], axis=1)


We also count the number of words in each text, since it has been observed from external analysis that the LLM models use less words than human (Closer to the essays prompt). We also divide by the maximum value to transform the attributes value to [0,1] interval.

In [13]:
ls = []
te = []
for i in range(len(dt)):
    x = re.sub(r"[!.?;\n,\"\']","",dt['text'][i].lower())
    te.append(x)
    ls.append(len(x.split()))
dt.text = te
ls = [i/max(ls) for i in ls]
dt['number_of_words'] = ls

In [14]:
dt["label"].unique()

array([0, 1])

# Find Unique Words in Each Class

Here, we will identify and extract unique words present in each class of the text data. This can help in understanding the distinct characteristics of each class.

In [30]:
st = dt[dt.label == 0].sample(n = 5000, random_state=1)
chat = dt[dt.label == 1].sample(n = 5000, random_state=1)
human_word = set(''.join(st['text'].to_list()))
non_human_word = set(''.join(chat['text'].to_list())) - human_word
print(''.join(non_human_word))
del dt
dt = pd.concat([st,chat]).sample(frac=1, random_state=2).reset_index(drop=True)
del st
del chat

🏦🤯💭💫🌟🤪😭🍁🤷💰🎢😕🏄о🏽🧠🛣🍋😬āé👦👕а🧀🥘🇷🌳🏞🎥🥪💚🏯🤖💘🏼🐶…😅💼😌👫💅🚌🌃😂🐧🤢👩💖😊⏰🍔💥🐠🏻🍗❄🕺📰🏟🥤🌨🤞🍿💨🎮🚗🤒🔍🌞📞🍽🤕🙅🥗🚨🐢🛋🎅🥛🌱♂🍕🌲🍟🍞🛫🧡🌌🤗😓🦄📚🙄🐦🍰😎🕹🧘🧙🎤🕒🇸🛍–‘️🏀‍😷🔭😴р💦🤤🙋🚀🙃📊🏠🐴🦐🌄🚂🏋🚕😆👀🙏😉🌸💉👨🗣🐟🍷🐱🚫🔥🥦─🔑🐰🛠🥕😤🧖💁🌠🤘😍🙈🙌📺💡🧦🐕💪😩🔬🍝📉🥑😲📸🎯🏥📱👍🎹👧🌧🏈


# Split the dataset

In this part, I am spliting the data into training (80%) and validation set (20%)

In [32]:
from sklearn.model_selection import train_test_split
ar = np.asarray(dt)
del dt
X_train, X_validation, Y_train, Y_validation = train_test_split(ar[:,[0,2]],ar[:,1], test_size = 0.2, random_state = 2)
del ar
X = pd.DataFrame(X_train)
print(X)

                                                      0         1
0     i am a student and i believe we should transit...  0.115942
1     hey im like totally stoked to be writing this ...  0.206522
2     as technology grows people have more options t...  0.219203
3     in todays world it is of paramount importance ...  0.138889
4     luke had a really great time when he was with ...   0.18901
...                                                 ...       ...
7995  title: the advantages of limiting car usageliv...  0.257246
7996  first impressions are an important part of for...   0.09843
7997  the planets are a fasinating part of science t...  0.175121
7998  korea is a country that has gained a lot of po...   0.19686
7999  i do not think that the author supported his i...  0.222222

[8000 rows x 2 columns]


In [33]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import RandomUnderSampler
from sklearn.feature_extraction.text import TfidfVectorizer


# Applying Vectorizer

In this step, we will convert the text data into numerical features using a TF-IDF vectorizer. This transformation is necessary because machine learning models cannot work with raw text directly.


In [34]:
Y_train = Y_train.astype("int")
vec = TfidfVectorizer(stop_words = "english",ngram_range=(2,3),max_features=10000)
X1 = vec.fit_transform(X[0],Y_train).toarray()


In [41]:
print(len(Y_train[Y_train == 0]))
print(len(Y_train[Y_train == 1]))


4043
3957


In [42]:
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV


# Train and Test the Model

Finally, we will train a machine learning model using the vectorized text data and evaluate its performance on a test dataset. This step involves training the model, and then assessing its accuracy and other metrics such AUC score.


In [45]:
from sklearn import svm
clf1 = svm.SVC(probability=True)
clf1.fit(X1,Y_train)
pred = clf1.predict(X1)
print("SVM")
print(confusion_matrix(Y_train,pred))

SVM
[[4043    0]
 [   7 3950]]


In [46]:
Y_validation = Y_validation.astype("int")
X_val_df = pd.DataFrame(X_validation)
X_te = vec.transform(X_val_df[0]).toarray()


In [48]:
X_te

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
confusion_matrix(Y_validation, clf1.predict(X_te))

array([[ 955,    2],
       [  30, 1013]])

In [52]:
from sklearn.metrics import roc_auc_score

In [53]:
roc_auc_score(Y_validation, clf1.predict_proba(X_te)[:,1])

0.9984711731992455

In [54]:
test = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")

In [55]:
test

,id,prompt_id,text
0,0000aaaa,2,Aaa bbb ccc.
1,1111bbbb,3,Bbb ccc ddd.
2,2222cccc,4,CCC ddd eee.


In [56]:
ls = []
te = []
for i in range(len(test)):
    x = re.sub(r"[?.!;\n,\"\']","",test['text'][i].lower())
    te.append(x)
    ls.append(len(x.split()))
test["text"] = te

In [60]:
test_df = vec.transform(test.text).toarray()


In [62]:
test_df

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [65]:
import csv

pred = clf1.predict_proba(test_df)
with open('submission.csv', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=',')
    field = ["id", "generated"]
    writer.writerow(field)
    for i in range(len(test.id)):
        check = 0
        row = []
        for j in test.text[i].split():
            if j in non_human_word:
                check = 1
        row.append(test.id[i])
        if check == 1:
            row.append(1)
        else:
            row.append(pred[:,1][i])
        writer.writerow(row)